## 44. 通配符匹配
```
给定一个字符串 (s) 和一个字符模式 (p) ，实现一个支持 '?' 和 '*' 的通配符匹配。
'?' 可以匹配任何单个字符。
'*' 可以匹配任意字符串（包括空字符串）。
两个字符串完全匹配才算匹配成功。
说明:
s 可能为空，且只包含从 a-z 的小写字母。
p 可能为空，且只包含从 a-z 的小写字母，以及字符 ? 和 *。

示例 1:
输入:
s = "aa"
p = "a"
输出: false
解释: "a" 无法匹配 "aa" 整个字符串。

示例 2:
输入:
s = "aa"
p = "*"
输出: true
解释: '*' 可以匹配任意字符串。

示例 3:
输入:
s = "cb"
p = "?a"
输出: false
解释: '?' 可以匹配 'c', 但第二个 'a' 无法匹配 'b'。

示例 4:
输入:
s = "adceb"
p = "*a*b"
输出: true
解释: 第一个 '*' 可以匹配空字符串, 第二个 '*' 可以匹配字符串 "dce".

示例 5:
输入:
s = "acdcb"
p = "a*c?b"
输出: false
```

## dp
- 本题与「10. 正则表达式匹配」非常类似，但相比较而言，本题稍微容易一些。因为在本题中，模式 p中的任意一个字符都是独立的，即不会和前后的字符互相关联，形成一个新的匹配模式。因此，本题的状态转移方程需要考虑的情况会少一些。

- 在给定的模式 p中，只会有三种类型的字符出现：
    - 小写字母 a−z，可以匹配对应的一个小写字母；
    - 问号 ?，可以匹配任意一个小写字母；
    - 星号 *，可以匹配任意字符串，可以为空，也就是匹配零或任意多个小写字母。

- 其中「小写字母」和「问号」的匹配是确定的，而「星号」的匹配是不确定的，因此我们需要*枚举*所有的匹配情况。为了减少重复枚举，我们可以使用动态规划来解决本题。
- 我们用 $dp[i][j]$表示字符串 s 的前 i个字符和模式 p 的前 j 个字符是否能匹配。在进行状态转移时，我们可以考虑模式 p 的第 j个字符 $p_j$，与之对应的是字符串 s中的第 i 个字符 $s_i$。
- (1) 如果 $p_j$ 是小写字母，那么 $s_i$ 必须为*相同的小写字母*，状态转移方程为：$$dp[i][j]=(si 与 pj 相同)∧dp[i−1][j−1]$$，其中 $∧$ 表示逻辑与运算。也就是说，dp[i][j]为真，当且仅当 dp[i−1][j−1] 为真，并且 $s_i$ 与 $p_j$ 相同。

- (2) 如果 $p_j$ 是问号，那么对 $s_i$没有任何要求，状态转移方程为：$dp[i][j]=dp[i−1][j−1]$

- (3) 如果 $p_j$是星号，那么同样对 $s_i$  没有任何要求，但是星号可以匹配零或任意多个小写字母，因此状态转移方程分为两种情况，即使用或不使用这个星号：
    - $dp[i][j]=dp[i][j−1]∨dp[i−1][j]$
    - 其中 ∨表示逻辑或运算。如果我们不使用这个星号，那么就会从 $dp[i][j−1]$ 转移而来；如果我们使用这个星号，那么就会从 $dp[i−1][j]$ 转移而来。

### 细节

- 只有确定了边界条件，才能进行动态规划。在上述的状态转移方程中，由于 dp[i][j] 对应着 s 的前 i 个字符和模式 p 的前 j 个字符，因此所有的 $dp[0][j]$ 和 $dp[i][0]$ 都是边界条件，因为它们涉及到空字符串或者空模式的情况，这是我们在状态转移方程中没有考虑到的：

    - $dp[0][0]=True$，即当字符串 s 和模式 p 均为空时，匹配成功；
    - $dp[i][0]=False$，即空模式无法匹配非空字符串；
    - $dp[0][j]$ 需要分情况讨论：因为星号才能匹配空字符串，所以只有当模式 p 的前 j 个字符均为星号时，$dp[0][j]$才为真。

- 我们可以发现，$dp[i][0]$的值恒为假，$dp[0][j]$ 在 j 大于模式 p 的开头出现的星号字符个数之后，值也恒为假，而 $dp[i][j]$ 的默认值（其它情况）也为假，因此在对动态规划的数组初始化时，我们就可以将所有的状态初始化为 False，减少状态转移的代码编写难度。

最终的答案即为 dp[m][n]，其中 m 和 n分别是字符串 s和模式 p 的长度。需要注意的是，由于大部分语言中字符串的下标从 0 开始，因此 $s_i$ 
$p_j$分别对应着 s[i−1] 和 p[j−1]。

In [1]:
class Solution:
    def isMatch(self, s: str, p: str) -> bool:
        m = len(s)
        n = len(p)
        dp = [[False] * (n+1) for _ in range(m+1)]
        dp[0][0] = True
        for j in range(1, n+1):
            if p[j-1] == "*":
                dp[0][j] = True
            else:
                break
        for i in range(1, m+1):
            for j in range(1, n+1):
                if s[i-1] == p[j-1] or p[j-1] == "?":
                    dp[i][j] = dp[i-1][j-1]
                if p[j-1] == "*":
                    dp[i][j] = dp[i-1][j] or dp[i][j-1]
        return dp[-1][-1]

In [2]:
s = "acdcb"
p = "a*c?b"

In [4]:
Solution().isMatch(s, p) is False

True

In [5]:
s = "adceb"
p = "*a*b"

In [6]:
Solution().isMatch(s, p) is True

True